# Animating trips with pydeck 

- Pydeck documentation for `TripsLayers`: https://deckgl.readthedocs.io/en/latest/gallery/trips_layer.html

- Stackoverflow example for pydeck 
https://stackoverflow.com/questions/65865145/animating-tripslayer-in-deck-gl-with-python

- Example of Trip Layer with Deck
https://deck.gl/examples/trips-layer/

In [287]:
import geopandas as gpd
import pandas as pd
import pydeck as pdk
import time
import json

In [288]:
file = open("scraped_data/bus_positions.json")
bus_positions = json.load(file)
file.close()

bus_positions[0]["bustime-response"]["vehicle"][0]

{'vid': '7936',
 'tmstmp': '20240130 22:12',
 'lat': '41.730838775634766',
 'lon': '-87.61434173583984',
 'hdg': '359',
 'pid': 18414,
 'rt': '3',
 'des': 'Michigan/Chicago',
 'pdist': 6368,
 'dly': False,
 'tatripid': '393',
 'origtatripno': '251981024',
 'tablockid': 'N4  -793',
 'zone': ''}

In [289]:
# Parsing data
# Leave only one vehicle 
a_single_bus_journey = []
# for snapshots in bus_positions:
for response in bus_positions: 
    if "vehicle" in response["bustime-response"]:
        for bus in response["bustime-response"]["vehicle"]: 
            if bus["vid"] == "1259": 
                bus["coordinates"] = [bus["lon"], bus["lat"]]
                a_single_bus_journey.append(bus)

# a_single_bus_journey

In [290]:
df_bus = pd.read_json(json.dumps(a_single_bus_journey))

/tmp/ipykernel_48219/1625725814.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_bus = pd.read_json(json.dumps(a_single_bus_journey))


In [291]:
df_bus["geometry"] = gpd.points_from_xy(df_bus.lon, df_bus.lat)
gdf_bus = gpd.GeoDataFrame(df_bus, geometry='geometry')
# gdf_bus


In [292]:
all_buses = []

for response in bus_positions: 
    if "vehicle" in response["bustime-response"]:
        for bus in response["bustime-response"]["vehicle"]: 
            bus["coordinates"] = [bus["lon"], bus["lat"]]
            all_buses.append(bus)

df_buses = pd.read_json(json.dumps(all_buses))
df_buses["geometry"] = gpd.points_from_xy(df_buses.lon, df_buses.lat)
gdf_buses = gpd.GeoDataFrame(df_buses, geometry='geometry')
gdf_buses

/tmp/ipykernel_48219/1647738045.py:9: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_buses = pd.read_json(json.dumps(all_buses))


,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,origtatripno,tablockid,zone,coordinates,geometry
0,7936,20240130 22:12,41.730838775634766,-87.61434173583984,359,18414,3,Michigan/Chicago,6368,False,393,251981024,N4 -793,,"[-87.61434173583984, 41.730838775634766]",POINT (-87.61434 41.73084)
1,1359,20240130 22:11,41.73215103149414,-87.61438751220703,358,18414,3,Michigan/Chicago,6845,False,394,251981859,3 -708,,"[-87.61438751220703, 41.73215103149414]",POINT (-87.61439 41.73215)
2,1560,20240130 22:11,41.849849700927734,-87.6186294555664,334,18414,3,Michigan/Chicago,50057,False,391,251981919,4 -710,,"[-87.6186294555664, 41.849849700927734]",POINT (-87.61863 41.84985)
3,1299,20240130 22:12,41.86909866333008,-87.62401610729741,357,18414,3,Michigan/Chicago,58480,False,389,251981635,3 -757,,"[-87.62401610729741, 41.86909866333008]",POINT (-87.62402 41.86910)
4,8456,20240130 22:12,41.89386,-87.61999333333333,178,18415,3,95th Red Line,0,False,1080326,251980537,N4 -792,,"[-87.61999333333333, 41.89386]",POINT (-87.61999 41.89386)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,1474,20240130 22:49,41.75149040222168,-87.66355361938477,182,1600,9,95th,77864,False,10001944,252019054,9 -626,,"[-87.66355361938477, 41.75149040222168]",POINT (-87.66355 41.75149)
570,1451,20240130 22:50,41.74885377150316,-87.6634737454928,181,3637,9,74th,10507,False,88357974,252019160,9 -608,,"[-87.6634737454928, 41.74885377150316]",POINT (-87.66347 41.74885)
571,8057,20240130 22:50,41.73936595916748,-87.66307067871094,0,9372,9,Irving Park,7033,False,10001922,252017664,N9 -695,,"[-87.66307067871094, 41.73936595916748]",POINT (-87.66307 41.73937)
572,1575,20240130 22:50,41.86336090451195,-87.66625068301246,354,9372,9,Irving Park,53003,False,10001924,252017349,N9 -694,,"[-87.66625068301246, 41.86336090451195]",POINT (-87.66625 41.86336)


In [ ]:
gdf_buses["tmstmp"]

In [294]:
gdf_buses["unix_tmstmp"] = gdf_buses["tmstmp"].astype('datetime64[s]').astype('int')
gdf_buses["unix_tmstmp"] = gdf_buses["unix_tmstmp"] - min(gdf_buses["unix_tmstmp"] )
gdf_buses["unix_tmstmp"] 

gdf_buses["vid"].unique()

array([7936, 1359, 1560, 1299, 8456, 7951, 7969, 1293, 1324, 8529, 1237,
       8523, 1201, 1327, 1263, 1329, 1265, 7945, 1285, 1325, 1277, 8457,
       7910, 8434, 8276, 4330, 8476, 4019, 4180, 4107, 4036, 1728, 8261,
       1290, 1292, 1212, 7994, 1259, 8458, 8507, 7980, 8550, 1308, 7914,
       1255, 8008, 1474, 1451, 8043, 8225, 1575, 1524, 1024, 8077, 7925,
       8557, 8057])

In [308]:
# Visualization 
view_chicago = pdk.ViewState(latitude = 41.8781, longitude = -87.6298, zoom = 12)

my_layers = [
    # Bus Stops 
    pdk.Layer(
        type = "ScatterplotLayer", 
        data = gdf_buses, 
        pickable = True, 
        get_position = "geometry.coordinates", 
        color = "vid", 
        # get_fill_color = [255, 0, 0],
        radius_scale = 15
    )
]

chi = pdk.Deck(
    layers=my_layers, 
    initial_view_state=view_chicago, 
    map_style='light', 
)
chi.to_html("trips.html")


# Try to animate plot

In [346]:
gdf_buses["vid"].unique()

df_bus = gdf_buses[gdf_buses["vid"] == 1259]

# df_bus.columns

In [352]:
df_bus = pd.DataFrame(df_bus[["coordinates", "unix_tmstmp"]])

list_coordinates = list(df_bus["coordinates"])
list_tmstmp = list(df_bus["unix_tmstmp"])

data = [[list_coordinates, list_tmstmp]]
 
# Create the pandas DataFrame
df_bus_trail = pd.DataFrame(data, columns=['coordinates', 'tmstmp'])

df_bus_trail

,coordinates,tmstmp
0,"[[-87.643065, 41.749965], [-87.64430236816406,...","[60, 1740, 1800, 1860, 1920, 1980, 2040, 2100,..."


In [353]:

# Visualization 
view_chicago = pdk.ViewState(latitude = 41.8781, longitude = -87.6298, 
                             zoom = 12, 
                             bearing=0, pitch=45)


layer = pdk.Layer(
    "TripsLayer", 
    df_bus_trail, 
    get_path = "coordinates", 
    get_timestamps = "tmstmp",
    current_time = 2040,
    trail_length =  600, 
    rounded = True 
)

chi = pdk.Deck(
    layers=[layer], 
    initial_view_state=view_chicago, 
    map_style='light', 
)
chi.to_html("trips.html")


-----------------------------
# Documentation example

In [334]:
"""
TripsLayer
==========

Plot of a single vehicle trip within San Francisco, fading in from the origin.

Adapted from a deck.gl documentation example.
"""

import pydeck as pdk
import pandas as pd

TRIPS_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/sf.trips.json"  # noqa

df = pd.read_json(TRIPS_LAYER_DATA)

df


,waypoints
0,"[{'coordinates': [-122.39079879999997, 37.7664..."


In [335]:

df["coordinates"] = df["waypoints"].apply(lambda f: [item["coordinates"] for item in f])
df["timestamps"] = df["waypoints"].apply(lambda f: [item["timestamp"] - 1554772579000 for item in f])

df.drop(["waypoints"], axis=1, inplace=True)
print(type(df))

df

<class 'pandas.core.frame.DataFrame'>


,coordinates,timestamps
0,"[[-122.39079879999997, 37.7664413], [-122.3908...","[0, 9, 54, 92, 345, 402, 462, 563, 880, 1070, ..."


In [320]:
df["coordinates"]

0    [[-122.39079879999997, 37.7664413], [-122.3908...
Name: coordinates, dtype: object

In [312]:

layer = pdk.Layer(
    "TripsLayer",
    df,
    get_path="coordinates",
    get_timestamps="timestamps",
    get_color=[253, 128, 93],
    opacity=0.8,
    width_min_pixels=5,
    rounded=True,
    trail_length=600,
    current_time=500,
)

view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=45)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("trips_layer.html")